In [ ]:
from pyspark.sql import SparkSession 


spark = SparkSession.builder.appName("Cars_ETL").config("spark.jars", "/home/nader-muhammad/.local/share/DBeaverData/drivers/maven/maven-central/com.mysql/mysql-connector-j-8.2.0.jar").getOrCreate()


df = spark.read.csv(r'/home/nader-muhammad/Documents/Data Engineering/Big_Projects/Data_warehouse2_CarsData/The Data/car_prices.csv', header=True, inferSchema=True)
df.show(5, truncate=False)

### Staging Layer

In [ ]:
# df.write.format("jdbc").option("user", "dbeaver").option("password", "Dbeaver@123").option("url", "jdbc:mysql://localhost:3306/ETL_cars").option("dbtable", "unclean_cars_spark")\
#          .option("driver", "com.mysql.cj.jdbc.Driver").mode("append").save()

### ETL Processing

### Uniqueness

In [ ]:
df = df.drop_duplicates()

### Name_standarize(Consistency)

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql import functions as f 
from pyspark.sql.functions import regexp_replace, upper,lower,initcap

In [ ]:
df.select("make").distinct().show(1000)

In [ ]:
df = df.withColumn('make', initcap('make'))

In [ ]:
df = df.withColumn('make', regexp_replace('make', 'Vw', 'Volkswagen'))
df = df.withColumn('make', regexp_replace('make', 'Mercedes-b', 'Mercedes-Benz'))
df = df.withColumn('make', regexp_replace('make', 'Mercedes-benz', 'Mercedes-Benz'))
df = df.withColumn('make', regexp_replace('make', 'Rolls-royce', 'Rolls-Royce'))
df = df.withColumn('make', regexp_replace('make', 'Bmw', 'BMW')).withColumn('make', regexp_replace('make', 'Geo', 'GEO'))
df = df.withColumn('make', regexp_replace('make', 'Landrover', 'Range Rover')).withColumn('make', regexp_replace('make', 'Land Rover', 'Range Rover'))


In [ ]:
df.select("make").distinct().show(1000)

In [ ]:
df.select("model").distinct().orderBy('model').count()

In [ ]:
df = df.withColumn('model', initcap('model'))

In [ ]:
df.select('model').distinct().count()

In [ ]:
df.select('trim').distinct().count()

In [ ]:
df = df.withColumn('trim', initcap('trim'))

In [ ]:
df.select('trim').distinct().count()

In [ ]:
df.select('body').distinct().count()

In [ ]:
df.select('body').distinct().show(1000)

In [ ]:
df = df.withColumn('body', initcap('body'))

df = df.withColumn('body', regexp_replace('body','Suv','SUV')).withColumn('body', regexp_replace('body', 'Tsx Sport Wagon', 'TSX Sport Wagon'))

In [ ]:
df.select('body').distinct().count()

In [ ]:
df.select('body').distinct().show(100)

In [ ]:
df.select('transmission').distinct().show(10)

In [ ]:
df = df.withColumn('transmission', initcap('transmission'))

In [ ]:
df.select('transmission').distinct().show(5)

In [ ]:
from pyspark.sql.functions import col, substring, concat, lit

df = df.withColumn("vin", concat(
                                 substring(col("vin"), 1, 3), lit("-"),
                                 substring(col("vin"), 4, 3), lit("-"),
                                 substring(col("vin"), 7, 4), lit("-"),
                                 substring(col("vin"), 11, 999)))

In [ ]:
df.select('state').distinct().show(100)

In [ ]:
df.select('color').distinct().count()

In [ ]:
df.select('color').distinct().show(1000)

In [ ]:
import numpy as np

In [ ]:
df = df.withColumn('color', initcap('color'))

df = df.withColumn('color', regexp_replace('color','—', np.nan ))

In [ ]:
df.select('color').distinct().count()

In [ ]:
df.select('color').distinct().show(50)

In [ ]:
df.select('interior').distinct().show(20)
df.select('interior').distinct().count()


In [ ]:
df = df.withColumn('interior', initcap('interior'))
df = df.withColumn('interior', regexp_replace('interior', '—', np.nan))

In [ ]:
df.select('interior').distinct().show(20)
df.select('interior').distinct().count()

In [ ]:
df.select('seller').distinct().show(10000000)
df.select('seller').distinct().count()

In [ ]:
df = df.withColumn('seller', initcap('seller'))

In [ ]:
df.select('seller').distinct().show(10000000, truncate=False)

### Handling Nulls (completeness)

In [ ]:
from pyspark.sql.functions import col, count, when

df.select([count(when(col(c).isNull(), c)).alias(c)
           for c in df.columns]).show()

In [ ]:
df = df.na.fill({'make':'No Data', 'model':'No Model', 'transmission':'No info', 
            'color':'No Color', 'interior':'Missing'})

In [ ]:
df = df.na.fill({'trim':'No Value', 'body':'No Data', 'vin':'Not Specified','saledate':'No Date'})

In [ ]:
mean_c = df.select(f.mean('condition')).collect()[0][0]
mean_o = df.select(f.mean('odometer')).collect()[0][0]
mean_m = df.select(f.mean('mmr')).collect()[0][0]
mean_s  = df.select(f.mean('sellingprice')).collect()[0][0]

df = df.na.fill({'condition':mean_c,'odometer':mean_o, 'mmr':mean_m, 'sellingprice':mean_s })

In [ ]:
df.select('body').distinct().show(10000)

In [ ]:
df.select([count(when(col(c).isNull(), c)).alias(c)
            for c in df.columns]).show()

### Convert DataTypes(Validity)

In [ ]:
df.printSchema()

In [ ]:
 ### substring(col(""), The start , the number of charcters)

In [ ]:
df = df.drop(col('saledate'))

In [ ]:
df.write.format('jdbc').option('user', 'dbeaver').option('password', 'Dbeaver@123').option('dbtable', 'clean_cars_spark').option('url', 'jdbc:mysql://localhost:3306/ETL_cars')\
         .option("driver","com.mysql.cj.jdbc.Driver").mode('append').save()